In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np

In [ ]:
# Referencing file paths to Load 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read school and student data files and stores into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

In [ ]:
# Calculating total number of schools
tot_schools=school_data["School ID"].count()
# Calculating total number of students
tot_students=school_data["size"].sum()
# Calculating total budget
tot_budget="${:,.2f}".format(school_data["budget"].sum())

In [ ]:
# Combining two datasets into a single dataset.  
clean_school_data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"]).copy()
clean_school_data.head(2)

In [ ]:
# calculate average math score
avg_math_score=round(clean_school_data.math_score.mean(),6)
# calculate average reading score
avg_read_score=round(clean_school_data.reading_score.mean(),5)
# calculate percentage of passing math
perc_pass_math=round(len(clean_school_data.loc[clean_school_data['math_score']>=70])/tot_students,8)
# calculate percentage of passing reading 
perc_pass_read=round(len(clean_school_data.loc[clean_school_data['reading_score']>=70])/tot_students,8)
# calculate percentage of passing math and reading
perc_overall_pass=round(len(clean_school_data.loc[(clean_school_data['reading_score']>=70) & 
                                            (clean_school_data['math_score']>=70)])/tot_students,8)

# create a dataframe for District Summary
dist_summary_df=pd.DataFrame({"Total Schools":[tot_schools], "Total Students":[tot_students], "Total Budget":[tot_budget],
                "Average Math Score":[avg_math_score], "Average Reading Score":[avg_read_score],
                "% Passing Math":[perc_pass_math*100], "% Passing Reading":[perc_pass_read*100],
                "% Overall Passing":[perc_overall_pass*100]})
dist_summary_df

In [ ]:
# School Summary
school_names=clean_school_data.groupby(['school_name'])
# extracting school type from the grouped school names
school_type=school_names.type.unique()

# total students each school
tot_studin_school=school_names['Student ID'].count()
# total budget of each school
tot_school_budget=school_names['budget'].mean()
# per student budget of each school
per_stud_budget=tot_school_budget/tot_studin_school
# average math score of each school
avg_school_math_score=school_names['math_score'].mean()
# average reading score of each school
avg_school_read_score=school_names['reading_score'].mean()

# Passing score is 70 or greater
# Passing math percentage of each school
pass_math=clean_school_data.loc[clean_school_data['math_score']>=70]
math_school=pass_math.groupby(['school_name']).count()
school_students=clean_school_data['school_name'].value_counts()
math_perc_pass=(math_school.math_score/school_students)*100

# Passing reading percentage of each school
pass_read=clean_school_data.loc[clean_school_data['reading_score']>=70]
read_school=pass_read.groupby(['school_name']).count()
read_perc_pass=(read_school.reading_score/school_students)*100
# Percentage of students that passed math and reading
overall_pass=clean_school_data.loc[(clean_school_data['reading_score']>=70) & 
                                            (clean_school_data['math_score']>=70)]
overall_school=overall_pass.groupby(['school_name']).count()
overall_perc_pass=(overall_school['math_score'] & overall_school['reading_score'])/school_students*100

# create the school summary dataframe
school_summary = pd.DataFrame({
    "Student Count":tot_studin_school,
    "School Type":school_type, 
    "School Budget":tot_school_budget, 
    "Per Student Budget":per_stud_budget,
    "Ave Math Score":avg_school_math_score, 
    "Ave Reading Score":avg_school_read_score, 
    "% Passing Math":math_perc_pass, 
    "% Passing Reading":read_perc_pass,
    "Overall Passing Rate":overall_perc_pass})

scl_summary=school_summary.copy()
school_sumry=school_summary.copy()
scl_sumry=school_summary.copy()
school_summary.head()

In [ ]:
# Calculating top five performaing schools (by percentage of passing)
top_perform_schools = school_summary.sort_values('Overall Passing Rate', ascending=False)
top_perform_schools.head()

In [ ]:
# Calculating five worst performaing schools (by percentage of passing)
top_perform_schools = school_summary.sort_values('Overall Passing Rate')
top_perform_schools.head()

In [ ]:
# Calculating math score by grade
# Create series of 9th, 10th, 11th, 12th grade
ninth_grade=clean_school_data.loc[clean_school_data.grade == '9th']
tenth_grade=clean_school_data.loc[clean_school_data.grade == '10th']
eleventh_grade=clean_school_data.loc[clean_school_data.grade == '11th']
twelth_grade=clean_school_data.loc[clean_school_data.grade == '12th']

# Grouping combined data by school name
ninth_grade_avg = ninth_grade.groupby("school_name").mean()
tenth_grade_avg = tenth_grade.groupby("school_name").mean()
eleventh_grade_avg = eleventh_grade.groupby("school_name").mean()
twelth_grade_avg = twelth_grade.groupby("school_name").mean()

# Create math score grouped by school name based on grades with math score
school_math_score=pd.DataFrame({"9th":ninth_grade_avg.math_score, "10th":tenth_grade_avg.math_score,
                                "11th":eleventh_grade_avg.math_score, "12th":twelth_grade_avg.math_score})
school_math_score

In [ ]:
# Calculations exactly same as above math score by grade
# Create math score grouped by school name based on grades with reading score
school_reading_score=pd.DataFrame({"9th":ninth_grade_avg.reading_score, "10th":tenth_grade_avg.reading_score,
                                "11th":eleventh_grade_avg.reading_score, "12th":twelth_grade_avg.reading_score})
school_reading_score

In [ ]:
# Scores by school spending
# Bins and Labels are creating to cut data into different fragments
bins=[0, 583, 630, 645, 675]
# Labels for binning 
labels=['<$584', '$585-629', '$630-644', '$645-675']
# School spending per student formatting from the school_summary dataframe using cut function
school_sumry['School Spending(Per Student)']=pd.cut(school_sumry['Per Student Budget'], bins, labels=labels, 
                                         include_lowest=True, right=True)
# Dataframe is grouping by School Spending (per Student) and calculating using mean function
school_spending=school_sumry.groupby(by='School Spending(Per Student)').mean()
# New dataframe created for the desired output
school_spending=pd.DataFrame({"Average Math Score":round(school_spending['Ave Math Score'],2),
                             "Average Reading Score":round(school_spending['Ave Reading Score'],2),
                             "% Passing Math":round(school_spending['% Passing Math'],2),
                             "% Passing Reading":round(school_spending['% Passing Reading'],2),
                             "% Overall Passing":round(school_spending['Overall Passing Rate'],2)})
school_spending

In [ ]:
# Scores by school size
# Bins and Labels are creating to cut data into different fragments
bins=[0, 1000, 2000, 5000]
# Labels for binning 
labels=['Samll(<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']
# Scores by school size is formatting by scl_summary dataframe using cut function
scl_summary['School Size']=pd.cut(scl_summary['Student Count'], bins, labels=labels, 
                                         include_lowest=True, right=True)
# Dataframe is grouping by School Size and calculating the mean
school_size=scl_summary.groupby(by='School Size').mean()
# New dataframe created for the desired output
school_size=pd.DataFrame({"Average Math Score":school_size['Ave Math Score'],
                             "Average Reading Score":school_size['Ave Reading Score'],
                             "% Passing Math":school_size['% Passing Math'],
                             "% Passing Reading":school_size['% Passing Reading'],
                             "% Overall Passing":school_size['Overall Passing Rate']})
school_size

In [ ]:
# Scores by school type
# Bins and Labels are creating to cut data into different fragments
bins=[0, 2500, 5000]
# Labels for binning 
labels=['Charter', 'District']
# Scores by school type is formatting by scl_summary dataframe using cut function
scl_sumry['School Type']=pd.cut(scl_sumry['Student Count'], bins, labels=labels,
                               include_lowest=False, right=False) 
                                         
# Dataframe is grouping by School Type and calculating the mean
school_type=scl_sumry.groupby(by='School Type').mean()
# # New dataframe created for the desired output
school_type=pd.DataFrame({"Average Math Score":school_type['Ave Math Score'],
                             "Average Reading Score":school_type['Ave Reading Score'],
                             "% Passing Math":school_type['% Passing Math'],
                             "% Passing Reading":school_type['% Passing Reading'],
                             "% Overall Passing":school_type['Overall Passing Rate']})
school_type